In [25]:
from owlready2 import *
import rdflib
from rdflib import Namespace, URIRef
from src.utils import *

In [26]:
dataset_name = 'family'
#dataset_name = 'pizza'
#dataset_name = 'OWL2DL-1'

In [27]:
if dataset_name == 'family':
    uri = Namespace("http://www.co-ode.org/roberts/family-tree.owl#")
elif dataset_name == 'pizza':
    uri = Namespace("http://www.co-ode.org/ontologies/pizza/pizza.owl#")
elif dataset_name.startswith('OWL2DL-'):
    uri = Namespace("http://benchmark/OWL2Bench#")

**Step 1: Load ontology**

In [28]:
g = rdflib.Graph()
g.parse(f'datasets/{dataset_name}.owl') 
print(f'# Triples: {len(g)}')

# Triples: 5017


In [29]:
g_tbox = rdflib.Graph()
g_tbox.parse(f'datasets/{dataset_name}_TBOX.owl')  
print(f'# Triples: {len(g_tbox)}')

# Triples: 1492


**Step 2: Get all subject-resources (individuals)**

In [30]:
individual_id, individual_names, individual_names_dict = get_individuals(g)

subject_resources = [str(x).split('#')[-1] for x in individual_names]
print(f'# Subject-Resources: {len(subject_resources)}')

# Subject-Resources: 508


**Step 3: Generate g = description graph for each r**

In [31]:
for i, subject in enumerate(subject_resources):
    describe_graph = copy_graph(g_tbox)
    for triple in g.triples((None, None, URIRef(uri + subject))):
        describe_graph.add(triple) 
    for triple in g.triples((URIRef(uri + subject), None, None)):
        describe_graph.add(triple) 
    describe_graph.serialize(destination= f'MyJenaProject/input_family/{subject}.ttl', format="ttl")